Dependencies

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from selenium import webdriver
import pandas as pd
import pymongo

In [2]:
executable_path = {'executable_path':r'C:\Users\mharg\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

Scraping for Data

Nasa news

In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup = bs(html, 'lxml')
news_title = soup.find('div', class_= 'content_title').text
news_content = soup.find('div', class_= 'article_teaser_body')

#the news content has quotes and the text property makes it return with 'None'

JPL image

In [4]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
response = requests.get(url)
soup = bs(response.text, 'lxml')
image_substr =soup.find('a', class_ = 'button fancybox')['data-fancybox-href']
jpl_image = 'https://www.jpl.nasa.gov/'+image_substr


Mars weather

In [5]:
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)
soup = bs(response.text, 'lxml')
mars_weather = soup.find('p', class_='TweetTextSize').text

Mars fact table

In [6]:
url = 'https://space-facts.com/mars/'
df = pd.read_html(url)[0]
df.columns = ['Measure', 'Value']
df.set_index('Measure', inplace = True)


In [7]:
fact_table = df.to_html()
fact_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Measure</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

Hemisphere images

In [8]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'lxml')
items = soup.find_all('div', class_='item')

In [9]:
#base url
base_url = 'https://astrogeology.usgs.gov'
mars_hemis = []
#loop
for i in items:
    title = i.find('h3').text
    partial_img = i.find('a', class_='itemLink product-item')['href']
    browser.visit(base_url + partial_img)
    partial_img = browser.html
    soup = bs( partial_img, 'lxml')
    #larger images
    img_url = base_url + soup.find('img', class_='wide-image')['src']
    mars_hemis.append({'title' : title, 'img_url' : img_url})

Add all scraped vars to db


In [10]:
s_dict = {}

s_dict['news_title'] = news_title 
s_dict['news_content'] = news_content
s_dict['jpl_image'] = jpl_image
s_dict['mars_weather'] = mars_weather
s_dict['fact_table'] = fact_table
s_dict['mars_hemis'] =  mars_hemis


In [11]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client.planets_db
db.mars.drop()


In [12]:
db.mars.insert_one(s_dict)

--- END OF APP ----